In [32]:
# Install a pip package in the current Jupyter kernel
import sys
!{sys.executable} -m pip install pandas
!{sys.executable} -m pip install numpy

#import packages
import pandas as pd
import numpy as np
import datetime as dt

In [48]:
# Import data
data_crime = pd.read_csv(r'https://raw.githubusercontent.com/mariusgruenewald/pol_viol/main/crime_data_edited.csv')
data_share = pd.read_csv(r'https://raw.githubusercontent.com/mariusgruenewald/pol_viol/main/election_data_edited.csv')

In [49]:
# get into datetime format
data_crime['year'] = data_crime['date'].astype(str).str.split('.', expand=True)[2]
data_crime['month'] = data_crime['date'].astype(str).str.split('.', expand=True)[1]
data_crime['day'] = data_crime['date'].astype(str).str.split('.', expand=True)[0]
data_crime['date'] = pd.to_datetime(data_crime[['year', 'month', 'day']].astype(float), errors='ignore')

In [50]:
# Merge crime and candidate statistics with merge over city string
data_crime = data_crime[(data_crime['Land'] == 14) | (data_crime['Land'] == 8) | (data_crime['Land'] == 16)]
data_share['city'] = data_share['city'].str.replace(r'Stadt ', '')
data_share['city'] = data_share['city'].str.replace('Ä', 'A').str.replace('Ö', 'O').str.replace('Ü', 'U')

list_cities_share = data_share.sort_values('city')['city'].unique()
list_cities_crime = data_crime['city'].unique()

data_crime.rename({'Land':'state'}, axis=1, inplace=True)
data_crime['state'] = data_crime['state'].map({16:'TH', 8:'BW', 14:'SN'})

In [56]:
list_cities_share[0:50], list_cities_crime[0:50]

(array(['Aach', 'Aalen', 'Abstatt', 'Abtsbessingen', 'Abtsgmund',
        'Achberg', 'Achern', 'Achstetten', 'Adelberg', 'Adelmannsfelden',
        'Adelsheim', 'Adorf/Vogtl.', 'Affalterbach', 'Aglasterhausen',
        'Ahorn', 'Aichelberg', 'Aichhalden', 'Aichstetten', 'Aichtal',
        'Aichwald', 'Aidlingen', 'Aitern', 'Aitrach', 'Albbruck',
        'Albershausen', 'Albstadt', 'Aldingen', 'Alfdorf', 'Allensbach',
        'Alleshausen', 'Allmannsweiler', 'Allmendingen',
        'Allmersbach im Tal', 'Alpirsbach', 'Altbach', 'Altdorf',
        'Altenberg', 'Altenbeuthen', 'Altenburg', 'Altengottern',
        'Altenriet', 'Altensteig', 'Altheim', 'Altheim (Alb)',
        'Althengstett', 'Althutte', 'Altlussheim', 'Altmittweida',
        'Altshausen', 'Am Ohmberg'], dtype=object),
 array(['Aalen', 'Abstatt', 'Abtsbessingen', 'Abtsgmund', 'Achberg',
        'Achern', 'Achstetten', 'Adelberg', 'Adelmannsfelden', 'Adelsheim',
        'Adorf/Vogtl.', 'Affalterbach', 'Aglasterhausen', 'Ahls

In [51]:
data_share.rename({'year':'cycle_1'}, axis=1, inplace=True)
data_share['cycle_2'] = data_share['cycle_1']
data_share['cycle_3'] = data_share['cycle_1']

Merging over years and cities? How many different cutoffs? Directly on election day, one month before and three months before

Cut-off days for the three states: 05.26.2019, 04.26.2019, 02.26.2019

In [52]:
# creating different cycle variables in crime data to merge over
#
data_crime['cycle_1'] = 2014
data_crime.loc[data_crime['date'] > dt.datetime(2019, 5, 26, 0, 0), 'cycle_1'] = 2019

data_crime['cycle_2'] = 2014
data_crime.loc[data_crime['date'] > dt.datetime(2019, 4, 26, 0, 0), 'cycle_2'] = 2019

data_crime['cycle_3'] = 2014
data_crime.loc[data_crime['date'] > dt.datetime(2019, 2, 26, 0, 0), 'cycle_3'] = 2019

In [53]:
# Find 
list_double = pd.DataFrame()
for city in data_share['city'].unique():
    
    data_city = data_share[data_share['city'] == city]
    for party in data_city['party'].unique():
        
        data_party = data_city[data_city['party'] == party]
        for election in data_party['cycle_1'].unique():
            
            data_election = data_party[data_party['cycle_1'] == election]
            
            if len(data_election) > 1:
                
                # append to dataset of 
                list_double = list_double.append(data_election)
                

In [61]:
# drop cities that are not uniquely identifable
data_share = data_share[~data_share.city.isin(list_double.city)]
list_double_cities = data_share[data_share.city.isin(list_double.city)]['city'].unique()

In [63]:
# Generate crime_count variable for easier aggregation
data_crime['crime_count'] = 0
data_crime.loc[data_crime['crime'] != 0, 'crime_count'] = 1
data_crime.drop('Unnamed: 0', axis=1, inplace=True)

In [71]:
len(data_crime[~data_crime['background'].isna()])

20433

In [91]:
data_crime

,crime,city,law,date,background,suspects,party,state,year,month,day,cycle_1,cycle_2,cycle_3,crime_count
91,NaN,Aalen,NaN,NaT,NaN,NaN,NaN,BW,None,None,nan,2014,2014,2014,1
146,NaN,Abstatt,NaN,NaT,NaN,NaN,NaN,BW,None,None,nan,2014,2014,2014,1
147,NaN,Abtsbessingen,NaN,NaT,NaN,NaN,NaN,TH,None,None,nan,2014,2014,2014,1
148,Sachbeschadigung,Abtsgmund,303 StGB,2019-05-20,Nicht zuzuordnen,0.0,CDU,BW,2019,05,20,2014,2019,2019,1
149,Sachbeschadigung,Abtsgmund,303 StGB,2019-05-20,Nicht zuzuordnen,0.0,CDU,BW,2019,05,20,2014,2019,2019,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
60411,Sachbeschadigung,Zwickau,303 StGB,2018-12-03,Rechts,0.0,NaN,SN,2018,12,03,2014,2014,2014,1
60412,Sachbeschadigung,Zwickau,303 StGB,2019-01-11,Links,0.0,NaN,SN,2019,01,11,2014,2014,2014,1
60413,Bedrohung,Zwickau,241 StGB,2019-02-13,Nicht,0.0,NaN,SN,2019,02,13,2014,2014,2014,1
60414,NaN,Zwiefalten,NaN,NaT,NaN,NaN,NaN,BW,None,None,nan,2014,2014,2014,1


### Generate various measures of crime

In [93]:
# regular summarization across cities and parties
data_crime_base = data_crime.groupby(['city', 'party', 'cycle_1', 'state'], as_index=False).sum()
data_crime_base

,city,party,cycle_1,state,suspects,cycle_2,cycle_3,crime_count
0,Abtsgmund,CDU,2014,BW,0.0,4038,4038,2
1,Abtsgmund,SPD,2014,BW,0.0,2019,2019,1
2,Achern,AfD,2014,BW,0.0,6052,6057,3
3,Achern,AfD,2019,BW,0.0,4038,4038,2
4,Adelsheim,SPD,2014,BW,0.0,2019,2019,1
...,...,...,...,...,...,...,...,...
806,Zwickau,GRÜNE,2019,SN,0.0,8076,8076,4
807,Zwickau,SPD,2014,SN,1.0,2019,2019,1
808,Zwickau,Sonstige Partei,2014,SN,1.0,6057,6057,3
809,Zwickau,Sonstige Partei,2019,SN,1.0,2019,2019,1


In [ ]:
data_crime_lr = data_crime[(data_crime['background'] == 'Links') | (data_crime['background'] == 'Rechts') | (data_crime['background'].isna())]

### Generate aggregation of fem_share

In [92]:
data_share

,Unnamed: 0,city,party,Anzahl Bewerber,Prozent Frauen,darunter Frauen,cycle_1,state,cycle_2,cycle_3
0,0,Stuttgart,AfD,60.0,26.666667,16.0,2019,BW,2019,2019
1,1,Stuttgart,CDU,60.0,41.666667,25.0,2019,BW,2019,2019
2,2,Stuttgart,DIE LINKE,60.0,48.333333,29.0,2019,BW,2019,2019
3,3,Stuttgart,FDP,60.0,30.000000,18.0,2019,BW,2019,2019
4,4,Stuttgart,GRÜNE,60.0,50.000000,30.0,2019,BW,2019,2019
...,...,...,...,...,...,...,...,...,...,...
16848,16848,Neuhaus am Rennweg,GRÜNE,1.0,100.000000,1.0,2014,TH,2014,2014
16849,16849,Sonneberg,GRÜNE,7.0,14.285714,1.0,2014,TH,2014,2014
16850,16850,Rudolstadt,GRÜNE,10.0,40.000000,4.0,2014,TH,2014,2014
16851,16851,Saalfeld/Saale,GRÜNE,13.0,38.461538,5.0,2014,TH,2014,2014


In [42]:
# Merge crime with share data
data_base_1 = pd.merge(data_crime_high_agg, data_share, on=['cycle_1', 'city', 'party', 'state'], how='right', validate='m:1')[['city','party','cycle_1','state','crime_count','Anzahl Bewerber','Prozent Frauen','darunter Frauen']]


In [43]:
# fill in non-affected crime cities
for data in (data_base_1,data_base_2,data_base_3,data_base_4):
    data['crime_count'].fillna(0, inplace=True)

In [89]:
data_share_lr 

,Unnamed: 0,city,party,Anzahl Bewerber,Prozent Frauen,darunter Frauen,cycle_1,state,cycle_2,cycle_3
0,0,Stuttgart,AfD,60.0,26.666667,16.0,2019,BW,2019,2019
1,1,Stuttgart,CDU,60.0,41.666667,25.0,2019,BW,2019,2019
2,2,Stuttgart,DIE LINKE,60.0,48.333333,29.0,2019,BW,2019,2019
3,3,Stuttgart,FDP,60.0,30.000000,18.0,2019,BW,2019,2019
4,4,Stuttgart,GRÜNE,60.0,50.000000,30.0,2019,BW,2019,2019
...,...,...,...,...,...,...,...,...,...,...
16848,16848,Neuhaus am Rennweg,GRÜNE,1.0,100.000000,1.0,2014,TH,2014,2014
16849,16849,Sonneberg,GRÜNE,7.0,14.285714,1.0,2014,TH,2014,2014
16850,16850,Rudolstadt,GRÜNE,10.0,40.000000,4.0,2014,TH,2014,2014
16851,16851,Saalfeld/Saale,GRÜNE,13.0,38.461538,5.0,2014,TH,2014,2014


In [90]:
# regular summarization across cities and parties
data_crime_lr_agg = data_crime_lr.groupby(['city', 'background', 'cycle_1', 'state'], as_index=False).sum()
data_crime_lr_agg.drop(['suspects', 'cycle_2', 'cycle_3'], axis=1, inplace=True)
data_base_1 = pd.merge(data_crime_lr_agg, data_share, on=['cycle_1', 'city', 'state'], how='right', validate='m:1')[['city','background','cycle_1','state','crime_count','Anzahl Bewerber','Prozent Frauen','darunter Frauen']]
data_base_1

MergeError: Merge keys are not unique in right dataset; not a many-to-one merge

### What we have now:
* Panel data set in levels

### What we need:
* Difference in election shares

In [29]:
# Generate differences
data_panel_diff = data_base_1[data_base_1['cycle_1'] == 2014]
data_panel_diff.drop(['Anzahl Bewerber', 'Prozent Frauen', 'darunter Frauen'], axis=1, inplace=True)
data_panel_diff

C:\Users\mariu\Anaconda3\lib\site-packages\pandas\core\frame.py:4315: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


,city,party,cycle_1,state,crime_count
6444,Stuttgart,AfD,2014,BW,11.0
6445,Stuttgart,CDU,2014,BW,1.0
6446,Stuttgart,DIE LINKE,2014,BW,0.0
6447,Stuttgart,FDP,2014,BW,4.0
6448,Stuttgart,GRÜNE,2014,BW,0.0
...,...,...,...,...,...
16455,Neuhaus am Rennweg,GRÜNE,2014,TH,0.0
16456,Sonneberg,GRÜNE,2014,TH,0.0
16457,Rudolstadt,GRÜNE,2014,TH,0.0
16458,Saalfeld/Saale,GRÜNE,2014,TH,0.0


In [30]:
# drop units with only one observation

data_panel = pd.DataFrame()
for city in data_base_1['city'].unique():
    data_city = data_base_1[data_base_1['city'] == city]
    for party in data_city['party'].unique():
        data_party = data_city[data_city['party'] == party]
        
        if len(data_party) > 1:
            
            data_panel_diff.loc[
                (data_panel_diff['party'] == party) & (data_panel_diff['city'] == city),
                'fem_share_diff'] = data_base_1[(data_base_1['party'] == party) & (data_base_1['city'] == city) & (data_base_1['cycle_1'] == 2019)
                                               ]['Prozent Frauen'].values - data_base_1[(data_base_1['party'] == party) & (data_base_1['city'] == city) & (data_base_1['cycle_1'] == 2014)
                                                                                       ]['Prozent Frauen'].values

C:\Users\mariu\Anaconda3\lib\site-packages\pandas\core\indexing.py:1599: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = infer_fill_value(value)
C:\Users\mariu\Anaconda3\lib\site-packages\pandas\core\indexing.py:1676: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(ilocs[0], value, pi)
C:\Users\mariu\Anaconda3\lib\site-packages\pandas\core\indexing.py:1676: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_i

In [31]:
data_panel_diff['crime_count'] = data_panel_diff['crime_count'].fillna(0)
data_panel_diff.to_csv(r'C:\Users\mariu\Documents\pol_viol\pol_viol\data_election_crime_merged.csv')

C:\Users\mariu\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
